### Create the data

https://cs231n.github.io/convolutional-networks/
https://medium.com/apache-mxnet/multi-channel-convolutions-explained-with-ms-excel-9bbf8eb77108

In [1]:
import random
import torch
from torch import nn, optim
import math
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
from cycler import cycler
import matplotlib.style
import matplotlib as mpl
from mpl_toolkits import mplot3d 

nn.Module comes in handy while writing many DL model. For example when you are trying to code Maxout Network as defined in the paper [Maxout Networks] (https://arxiv.org/pdf/1302.4389.pdf 44).
https://github.com/pytorch/pytorch/commit/c7c8aaa7f040dd449dbc6aca9204b2f943aef477
https://discuss.pytorch.org/t/multiple-parallel-fully-connected-layers-type-torch-cuda-floattensor-but-found-type-torch-floattensor/37810
https://www.geeksforgeeks.org/single-neuron-neural-network-python/
https://rhettinger.wordpress.com/2011/05/26/super-considered-super/


In [2]:
%matplotlib notebook
import glob, os, os.path
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import stats
import bisect
import scipy.sparse as sparse  #for baseline subtraction
from matplotlib import rc
# rc('mathtext', default='regular')
import h5py

In [3]:
hfont = {'fontname':'Helvetica'}

In [4]:
'''Definitions'''

def gaussian(x, x0, r, a, b):
    y = b + a*np.exp(-(x-x0)**2/(2*r**2))
    return y

def PseudoVoigtFunction(WavNr, Pos, Amp, GammaL, FracL):
    SigmaG = GammaL / np.sqrt(2*np.log(2)) # Calculate the sigma parameter  for the Gaussian distribution from GammaL (coupled in Pseudo-Voigt)
    LorentzPart = Amp / np.pi * (GammaL / ((WavNr - Pos)**2 + GammaL**2)) # Lorentzian distribution
    GaussPart = Amp / (np.sqrt(2*np.pi) * SigmaG) * np.exp( -(WavNr - Pos)**2 / (2 * SigmaG**2)) # Gaussian distribution
    Fit = FracL * LorentzPart + (1 - FracL) * GaussPart # Linear combination of the two parts (or distributions)
    return Fit

#baseline subtraction based on Asymetric Least Square Smoothing
#https://zanran_storage.s3.amazonaws.com/www.science.uva.nl/ContentPages/443199618.pdf
def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = sparse.csc_matrix(np.diff(np.eye(L), 2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = sparse.linalg.spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z

# Function which extracts the number of all XYfiles in the directory.    
run_no=0
def get_number_XYfiles(run_no):
    directory = os.listdir('data/')
    number_XYfiles = run_no
    for Dir in directory:
        number_XYfiles = number_XYfiles+1
    return(number_XYfiles)

In [5]:
# show the number of XYfiles found in the directory
total_XYfiles = get_number_XYfiles(run_no)
print(total_XYfiles)
total_XYfiles=350 #[Pt=1084, MgFeOB1_263]
print('The number of XYfiles within the chosen run is:', total_XYfiles)
%pwd
%mkdir -p ClassificationFigures
save_figures_to = '../ClassificationFigures/'

350
The number of XYfiles within the chosen run is: 350


In [ ]:
'''make sure that you have the right command directory selected'''
%cd /gpfs/exfel/data/user/sunyue/Downloads/Brockhauser_Sandor/L2_Beamtime2019_Fe200culet_fe80mg20/XY/
theta = [] 
I = []
baseline = []
Icorrect = []

test_y=total_XYfiles-2
for run_no in range(1,0+total_XYfiles):
    fileNO = str(run_no).zfill(4) 
    fname = 'l2_fe80mg20_ramp_00001_m1_'+str(run_no).zfill(4)+'.xy'
    thetas, Is = np.loadtxt(fname, skiprows=25, unpack=True) # reads all files in range and assigns 1st column to thetas and 2nd to Is
    I.append(Is) # attaches a new array each time to a copy of previous, not really necessary here
    idx1 = 0
    idx2 = 4050
#background subtraction
#parameters lam and p have to be adjusted by hand!
#try 10**2 < lam < 10**9
#    0.001 <  p  < 0.1
    lam = 100000
    p = 0.002
    baselines = baseline_als(Is[idx1:idx2], lam, p)
    baseline.append(baselines)
    Icorrected = Is[idx1:idx2] - baselines
    Icorrect.append(Icorrected)
#    if (run_no==15): 
#        print(thetas.size, Is.shape,type(I),np.shape(I),len(Icorrected),len(baselines),sep='\n')
    thetas = thetas[idx1:idx2]
    theta.append(thetas)

/gpfs/exfel/data/user/sunyue/Downloads/Brockhauser_Sandor/L2_Beamtime2019_Fe200culet_fe80mg20/XY


In [ ]:
%cd -

In [ ]:
print(theta[0].shape,np.size(theta),len(theta),np.size(theta[0]))

In [ ]:
349*4023

In [ ]:
plt.figure()
plt.plot(theta[98], I[98][idx1:idx2])
plt.plot(theta[98], baseline[98],color = 'limegreen')
plt.plot(theta[98], Icorrect[98], color = 'darkred')
#plt.plot(theta[0], I[0], color = 'red')
#plt.plot(theta[1500], I[1500][idx1:idx2])
#plt.plot(theta[1500], baseline[1500])
#plt.plot(theta[450], Icorrect[450], color = 'limegreen')
#plt.plot(theta[420], I[420], color = 'limegreen')
plt.ylabel('Intensity')#, fontsize = 12
plt.xlabel('Diffraction angle 2$\Theta$ (deg.)')#, fontsize = 12
plt.tight_layout

In [ ]:
def minibatch_plot_data(X, y, ai1=-1,ai2=1,bi1=-1,bi2=1,d=0, auto=False, zoom=1):
    X = X.cpu()
    y = y.cpu()
#     plt.scatter(X.numpy()[:, 0], X.numpy()[:, 1], c=y, s=2, cmap=plt.cm.rainbow)
    plt.scatter(X.numpy()[:, 0], X.numpy()[:, 1], c=y, s=2, cmap=plt.cm.rainbow)
#     plt.axis('square')
#     plt.axis(np.array((ai1, ai2, bi1, bi2)) * zoom)
#     if auto is True: plt.axis('equal')
    plt.axis('on') 
    
def minibatch_plot_model(mndinx1,mndinx2,mndiny1,mndiny2,X2, y2, model):
    model.cpu()
#     nx, ny = (200, 200)
#     x = np.linspace(mndinx1, mndinx2, nx)
#     y = np.linspace(mndiny1,mndiny2, ny)
    x = np.arange(mndinx1, mndinx2, 0.02)
    y = np.arange(mndiny1, mndiny2, 0.001)
    xx, yy = np.meshgrid(x, y)
    with torch.no_grad():
        data = torch.from_numpy(np.vstack((xx.reshape(-1), yy.reshape(-1))).T).float()
        Z = model(data).detach()
    Z = np.argmax(Z, axis=1).reshape(xx.shape)
    
    plt.contourf(xx, yy, Z, cmap=plt.cm.rainbow, alpha=0.3)
    minibatch_plot_data(X2, y2,mndinx1,mndinx2,mndiny1,mndiny2,zoom=1)

In [ ]:
def plot_data(X, y, d=0, auto=False, zoom=1):
    X = X.cpu()
    y = y.cpu()
#     plt.figure(figsize=(8,8))
    ax = plt.subplot(111)
    ax.set_ylabel('Intensity')#, fontsize = 16,
    ax.set_xlabel('Diffraction angle 2$\Theta$ (deg.)')#, fontsize = 16
#     plt.axis(np.array((axl, axr, byl, byh)) * zoom)
    
#     plt.rc('xtick',labelsize=16)
#     plt.rc('ytick',labelsize=16)
    plt.scatter(X.numpy()[:, 0], X.numpy()[:, 1], c=y, s=0.6, cmap=plt.cm.rainbow)

In [ ]:
def set_default(figsize=(8, 8)):
#     plt.style.use(['dark_background', 'bmh'])
#     plt.rc('figure', facecolor='none')
    plt.rc('figure', figsize=figsize)
    
def yue_plot_data(X, y, axl,axr,byl,byh,d=0, auto=False, zoom=1):
    X = X.cpu()
    y = y.cpu()
#     plt.figure(figsize=(8,8))
    ax = plt.subplot(111)
    ax.set_ylabel('Intensity')#, fontsize = 16,
    ax.set_xlabel('Diffraction angle 2$\Theta$ (deg.)')#, fontsize = 16
    plt.axis(np.array((axl, axr, byl, byh)) * zoom)
    #ax.set_ylim(-2,50)
    #ax.set_xlim(9.5,26)
    
#     plt.rc('xtick',labelsize=16)
#     plt.rc('ytick',labelsize=16)
    plt.scatter(X.numpy()[:, 0], X.numpy()[:, 1], c=y, s=0.6, cmap=plt.cm.rainbow)
    #plt.axis('auto')      
#     ax.spines['right'].set_visible(False) #remove right axis spine
#     ax.spines['top'].set_visible(False) # remove top axis spine  
    
    """
    X = X.cpu()
    y = y.cpu()
    plt.axis(np.array((axl, axr, byl, byh)) * zoom)
    if auto is True: plt.axis('equal')
    plt.axis('off')

    plt.tight_layout()
    #_m, _c = 0, '.15'
    #plt.axvline(0, ymin=_m, color=_c, lw=1, zorder=0)
    #plt.axhline(0, xmin=_m, color=_c, lw=1, zorder=0)
    """
    """
    cmaps = [('Perceptually Uniform Sequential', [
                'viridis', 'plasma', 'inferno', 'magma', 'cividis']),
             ('Sequential', [
                'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
                'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
                'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']),
             ('Sequential (2)', [
                'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
                'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
                'hot', 'afmhot', 'gist_heat', 'copper']),
             ('Diverging', [
                'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
                'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic']),
             ('Cyclic', ['twilight', 'twilight_shifted', 'hsv']),
             ('Qualitative', [
                'Pastel1', 'Pastel2', 'Paired', 'Accent',
                'Dark2', 'Set1', 'Set2', 'Set3',
                'tab10', 'tab20', 'tab20b', 'tab20c']),
             ('Miscellaneous', [
                'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
                'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
                'gist_rainbow', 'rainbow', 'jet', 'nipy_spectral', 'gist_ncar'])]
    """

def ys_plot_model(X, y, model):
    plt.figure(figsize=(8,8))
    bx = plt.subplot(111)
    plt.axis([axl, axr, byl, byh])
    
#     plt.rc('xtick',labelsize=16)
#     plt.rc('ytick',labelsize=16)
    
    model.cpu()
    X=X.cpu()
    mesh1 = np.arange(axl, axr, 0.01)
    mesh2 = np.arange(byl, byh, 0.01)
    xx, yy = np.meshgrid(mesh1, mesh2)
    
    with torch.no_grad():
        data = torch.from_numpy(np.vstack((xx.reshape(-1), yy.reshape(-1))).T).float().cpu()
        Z = model(data).detach()
        #print(xx,Z.shape,Zt,data.shape)
    Z = np.argmax(Z, axis=1).reshape(xx.shape)

    plt.contourf(xx, yy, Z, cmap=plt.cm.rainbow, alpha=0.3)     
    yue_plot_data(X, y, axl,axr,byl,byh)
    
    
def yue_plot_3D(X, X_axis):
    X = X.cpu()
    X_axis = X_axis.cpu()    
    
    fig = plt.figure(figsize = (10, 7)) 
    ax = plt.axes(projection ="3d") 

    # Creating plot 
    ax.scatter3D(X_axis.numpy(), X.numpy()[:, 0], X.numpy()[:, 1],s=0.3, color = "green"); 
    
    plt.title("Spectra 3D scatter plot") 
    ax.set_xlabel('X-axis') # , fontweight ='bold'
    ax.set_ylabel('angle')  #, fontweight ='bold'
    ax.set_zlabel('intensity') #, fontweight ='bold'

    # show plot 
    plt.show() 

In [ ]:
set_default()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

###  data for training

In [ ]:
# data for training
#plotting various plots to show development of bcc -> hcp transition
plt.figure(figsize=(8,8))

# plt.rc('xtick',labelsize=16)
# plt.rc('ytick',labelsize=16)

# These are the colors that will be used in the plot
color_sequence = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c',
                  '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',
                  '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f',
                  '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']

ax = plt.subplot(111)
N=len(theta[0])

X_1 = []# training: theta
X_2 = []# training: intensity
y   = []
global Ci,Ci_0,div_num,gap #indicates num of curves, and curves of class0.
div_num = 185 #indicates the boundary of spectral curves.
gap = 1 #20
Ci = 0  
Ci_0 = 0
X_axis=[] # used to store data for 3D plotting

for pct in range(0,total_XYfiles-1,gap):
    #ax.plot(theta[pct], Icorrect[pct]+6-pct/50., color = 'limegreen' if pct <200 else 'blue')
    #ax.plot(theta[pct], Icorrect[pct]+6-pct/50., color = color_sequence[Ci])
#     ax.plot(theta[pct], Icorrect[pct]+6-pct/50.,lw=1)
    ax.plot(theta[pct], Icorrect[pct],lw=0.6)
 
    #testX = torch.FloatTensor(theta[pct])
    #testX = torch.cat((testX),1)
    
    Icorrect_t = Icorrect[pct]
#     Icorrect_t = Icorrect[pct]+6-pct/50.
    theta_t = theta[pct]
    X_1.append(theta_t)
    X_2.append(Icorrect_t)
    X_axis.append(np.ones(N)*pct) 
    
    Ci += 1
    if pct <div_num:
        Ci_0 += 1
    #y.append(torch.zeros(1,size(theta[pct])) if pct < 200 else torch.ones(1,size(theta[pct]))

             
print(X_1[0].shape,Ci,Ci_0)


ax.set_ylabel('Intensity')#, fontsize = 16,
ax.set_xlabel('Diffraction angle 2$\Theta$ (deg.)')#, fontsize = 16
# ax.set_ylim(-2,45)
ax.set_xlim(9.5,26)

plt.title('[Fe10,Mg90]O 5s ramp, 100 ms exposure')#, fontsize=16, fontweight='bold'

# ax.spines['right'].set_visible(False) #remove right axis spine
# ax.spines['top'].set_visible(False) # remove top axis spine
plt.tight_layout()
plt.savefig(save_figures_to+"Original data used for training.png", bbox_inches='tight', dpi=600)

In [ ]:
(np.size(X_1)), len(X_1), 349*4023

In [ ]:
#plotting spectra in another way
plt.figure(figsize=(8,8))
# plt.rc('xtick',labelsize=16)
# plt.rc('ytick',labelsize=16)

ax = plt.subplot(111)
t_gap=20 # this parameter decide the number of spectral curves for test.

import matplotlib.cm as cm
colors = cm.brg(np.linspace(0, 1, len(range(0,total_XYfiles-1,t_gap)))) # or gist_rainbow

pos_d=0
for pct,cc in zip(range(0,total_XYfiles-1,t_gap),colors):
    pct = pct+pos_d
    plt.scatter(theta[pct], Icorrect[pct], color=cc,s=0.3)
    
ax.set_ylabel('Intensity')
ax.set_xlabel('Diffraction angle 2$\Theta$ (deg.)')
ax.set_ylim(-2,50)
ax.set_xlim(9.5,26)
plt.grid(True)
plt.title('[Fe10,Mg90]O 5s ramp, 100 ms exposure')#, fontsize=16, fontweight='bold'
plt.savefig(save_figures_to+"Spectra data for testing (manually classified).png", bbox_inches='tight', dpi=600)

plt.tight_layout()


In [ ]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# # X = sc.fit(X_2)
# X_i = sc.fit_transform(X_2)
# # X_train = sc.fit_transform(X_2)
# # X_test = sc.transform(X_test)

In [ ]:
standard_flag = 0
if standard_flag:
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_i = sc.fit_transform(X_2)
    # X = sc.fit(X_2)
    # X_train = sc.fit_transform(X_2)
    # X_test = sc.transform(X_test)
else:
    X_i = X_2 - np.mean(X_2,axis=0)

In [ ]:
print(X_i.mean(axis=0))
print(X_i.std(axis=0))

In [ ]:
PCA_NUM_COMP = len(X_i)-1
print(PCA_NUM_COMP)

In [ ]:
from sklearn.decomposition import PCA
Sum_Explained_variance =.99
pca = PCA(Sum_Explained_variance)
# pca = PCA(n_components=240)
X2_data =pca.fit_transform(X_i)
# X2_test = pca.transform(X_test)
N_Compents=pca.n_components_ 
print(X2_data.shape)
print(N_Compents)

In [ ]:
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(np.cumsum(pca.explained_variance_ratio_),'o-')
plt.xlabel('number of components');
plt.ylim(0,1.0)
plt.ylabel('cumulative explained variance');

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(X2_data,'.-')
plt.xlabel('number of components');
print(X2_data.shape)
plt.figure(figsize=(6,6))
print(X2_data[-3:-1].shape)
plt.plot(X2_data[100],'.-')

In [ ]:
type(X2_data)

# pick certern extreme curves and add some random noise to train the model

In [ ]:
N = N_Compents

# p_num=[0,32 ,8,26, 14,30, 2,34, 3,31, 4,21, 10,23, 5,22, 7,27, 15,28, 16,29, 9,24] 
p_num_c0= [i for i in range(16)] #[0,2,4,5,7,8,10,15] #1 #5  class 0 class 1 class 0
p_num_c1= [i for i in np.arange(22,34)] #[22,24,28,30,32,34] #1 #5  class 0 class 1 class 0
p_num = p_num_c0 + p_num_c1
print(p_num,len(p_num_c1))
N_TrainOrg = len(p_num)
Train_Gap = 10
X_2=torch.FloatTensor(X2_data)
Xtemp = torch.zeros(len(p_num),N_Compents)
X_Extrem_tmp = torch.empty(0,N_Compents)

plt.figure(figsize=(8,6))
ax = plt.subplot(111)

for i in range(len(p_num)):
    Xtemp[i] = X_2[Train_Gap*p_num[i]] # intensity
    ax.plot(Xtemp[i,1:20]-i*7,lw=0.6,color='red' if Train_Gap*p_num[i]<185 else 'blue')
    print(X_Extrem_tmp.shape,Xtemp[i].shape)
    X_Extrem_tmp=torch.cat((X_Extrem_tmp,torch.reshape(Xtemp[i],(1,-1))),0)
print(Xtemp.shape)

X_Extrem= torch.empty(0,N_Compents)
X_Extrem    =torch.cat((X_Extrem,X_Extrem_tmp),0) #  spectral curve are stacked in one column.


N_sim=100 # number of simulated spectra for each original ones


In [ ]:
# """
N = N_Compents

# p_num=[0,32 ,8,26, 14,30, 2,34, 3,31, 4,21, 10,23, 5,22, 7,27, 15,28, 16,29, 9,24] 
# p_num_c0= [i for i in np.arange(1,16)] #[0,2,4,5,7,8,10,15] #1 #5  class 0 class 1 class 0
p_num_c0= [i for i in range(16)] #[0,2,4,5,7,8,10,15] #1 #5  class 0 class 1 class 0
p_num_c1= [i for i in np.arange(22,34)] #[22,24,28,30,32,34] #1 #5  class 0 class 1 class 0
p_num = p_num_c0 + p_num_c1
print(p_num, len(p_num_c0),len(p_num_c1))
N_TrainOrg = len(p_num)
Train_Gap = 10
X_2=torch.FloatTensor(X2_data)
Xtemp = torch.zeros(len(p_num),N_Compents)
X_Extrem_tmp = torch.empty(0,N_Compents)

plt.figure(figsize=(6,4))
ax = plt.subplot(111)

for i in range(len(p_num)):
    Xtemp[i] = X_2[Train_Gap*p_num[i]] # intensity
    if i==0: 
        ax.plot(Xtemp[i],lw=0.6,color='red' if i< len(p_num_c0) else 'blue', label='Class 0')#
    elif i==len(p_num)-1:
        ax.plot(Xtemp[i],lw=0.6,color='red' if i< len(p_num_c0) else 'blue', label='Class 1')#
    else:
        ax.plot(Xtemp[i],lw=0.6,color='red' if i< len(p_num_c0) else 'blue')
        
    print(X_Extrem_tmp.shape,Xtemp[i].shape)
    X_Extrem_tmp=torch.cat((X_Extrem_tmp,torch.reshape(Xtemp[i],(1,-1))),0)
print(Xtemp.shape)

X_Extrem= torch.empty(0,N_Compents)
X_Extrem    =torch.cat((X_Extrem,X_Extrem_tmp),0) #  spectral curve are stacked in one column.

# plt.figure(figsize=(8,6))
# ax = plt.subplot(111)
# ax.plot(X_Extrem[0],lw=0.6,color='red')
# ax.plot(X_Extrem[-1],lw=0.6,color='blue')

N_sim=100 # number of simulated spectra for each original ones

for n in range(N_sim):
    X_Extrem_tmp = torch.empty(0,N_Compents)
    seed = n*1000
    random.seed(seed)
    torch.manual_seed(seed)
    for j in range(len(p_num)):
        for i in range(N_Compents):
            Xtemp[j,i] = Xtemp[j,i] +random.random()/12
        ax.plot(Xtemp[j],lw=0.6,color='red' if j< len(p_num_c0) else 'blue')
        X_Extrem_tmp=torch.cat((X_Extrem_tmp,torch.reshape(Xtemp[j],(1,-1))),0)
    X_Extrem    =torch.cat((X_Extrem,X_Extrem_tmp),0) #  spectral curve are stacked in one column.
#     y_Extrem_tmp=torch.cat((ytemp1,ytemp2),0)
#     y_Extrem    =torch.cat((y_Extrem,y_Extrem_tmp),0)
# one row for one spectral curve:
print((X_Extrem).shape)

# plt.tight_layout()


N_Train=N_TrainOrg*N_sim+N_TrainOrg

plt.xlabel('PCs');
plt.ylabel('intensity (after PCA processing)');
plt.title('Training dataset');
plt.legend()
plt.grid(True)
plt.savefig(save_figures_to+"Training dataset.svg", bbox_inches='tight', dpi=300)

#  ===============================method 2============================
global X2,y2  
X2 = X_Extrem.to(device)
y2_orig = torch.cat((torch.zeros(len(p_num_c0),1),torch.ones(len(p_num_c1),1)),0)
y2 = torch.empty(0)
for n in range((N_sim+1)):
    y2= torch.cat([y2, y2_orig], dim=0) 

y2 = torch.reshape(y2,(-1,1)).to(device)
print(X2.shape,y2.shape,type(y2))

In [ ]:
### training data selected in the original dataset.

N_orig = len(theta[0])

fig, ax = plt.subplots(figsize=(6,6))
# plt.rc('xtick') #,labelsize=16
# plt.rc('ytick') #,labelsize=16

# These are the colors that will be used in the plot
color_sequence = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c',
                  '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',
                  '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f',
                  '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']


# X_1 = []# training: theta
# X_2 = []# training: intensity
y   = []
global Ci,Ci_0,div_num,gap #indicates num of curves, and curves of class0.
div_num = 185 #indicates the boundary of spectral curves.
gap = 10
# Ci = 0  
# Ci_0 = 0
for pct in range(0,total_XYfiles,gap):
    shift = 12-pct/5.
    ax.plot(theta[pct][0:N_orig], Icorrect[pct][0:N_orig]+shift, color = 'red' if ((pct in [gap*l for l in p_num_c0])) else ('blue' if (pct in [gap*l for l in p_num_c1]) else 'limegreen')  ,lw=0.6)
    Ci += 1
    if pct <div_num:
        Ci_0 += 1
    
    Icorrect_t = Icorrect[pct]
    theta_t = theta[pct]
#     X_1.append(theta_t[0:N_orig])
#     X_2.append(Icorrect_t[0:N_orig])
    if not pct % 20:
        ax.text(theta_t[int(2*N_orig/3)], Icorrect_t[int(2*N_orig/3)]+shift, "n = %i" % (pct) , size=6, ha="center", color="k")
             
# print(X_1[0].shape,Ci,Ci_0)


ax.set_ylabel('Intensity', )
ax.set_xlabel('Diffraction angle 2$\Theta$ (deg.)',)
ax.set_xlim(9.5,26)
plt.grid(True)
plt.title('[Mg0.2,Fe0.8] 50s ramp, 100 ms exposure') #,  fontweight='bold'
ax.get_yaxis().set_ticks([])
# plt.rc('xtick',labelsize=6)
# plt.rc('ytick',labelsize=16)
    
plt.tight_layout()
plt.savefig(save_figures_to+"Original data used for training.svg")

In [ ]:
# # """
# N = N_Compents

# # p_num=[0,32 ,8,26, 14,30, 2,34, 3,31, 4,21, 10,23, 5,22, 7,27, 15,28, 16,29, 9,24] 
# p_num=[0,32 ,8,26, 14,30, 2,34, 3,31, 4,21, 10,23, 5,22, 7,27, 15,28] #1 #5  class 0 class 1 class 0
# N_TrainOrg = len(p_num)
# Train_Gap = 10
# X_2=torch.FloatTensor(X2_data)
# Xtemp = torch.zeros(len(p_num),N_Compents)
# X_Extrem_tmp = torch.empty(0,N_Compents)
# for i in range(len(p_num)):
#     Xtemp[i] = X_2[Train_Gap*p_num[i]] # intensity
#     print(X_Extrem_tmp.shape,Xtemp[i].shape)
#     X_Extrem_tmp=torch.cat((X_Extrem_tmp,torch.reshape(Xtemp[i],(1,-1))),0)
# print(Xtemp.shape)

# X_Extrem= torch.empty(0,N_Compents)
# X_Extrem    =torch.cat((X_Extrem,X_Extrem_tmp),0) #  spectral curve are stacked in one column.

# plt.figure(figsize=(8,6))
# ax = plt.subplot(111)
# ax.plot(Xtemp[0],lw=0.6,color='red')
# ax.plot(Xtemp[2],lw=0.6,color='blue')
# N_sim=200 # simulated spectra 

# for n in range(N_sim):
#     X_Extrem_tmp = torch.empty(0,N_Compents)
#     seed = n*1000
#     random.seed(seed)
#     torch.manual_seed(seed)
#     for j in range(len(p_num)):
#         for i in range(N_Compents):
#             Xtemp[j,i] = Xtemp[j,i] +random.random()/12
#         ax.plot(Xtemp[j],lw=0.6,color='red' if j/2==0 else 'blue')
#         X_Extrem_tmp=torch.cat((X_Extrem_tmp,torch.reshape(Xtemp[j],(1,-1))),0)
#     X_Extrem    =torch.cat((X_Extrem,X_Extrem_tmp),0) #  spectral curve are stacked in one column.
# #     y_Extrem_tmp=torch.cat((ytemp1,ytemp2),0)
# #     y_Extrem    =torch.cat((y_Extrem,y_Extrem_tmp),0)
# # one row for one spectral curve:
# print((X_Extrem).shape)

# plt.tight_layout()






# ### ====================assign labels to training set============================
# N_Train=N_TrainOrg*N_sim+N_TrainOrg
# global X2,y2  
# X2 = X_Extrem.to(device)
# # y2 = torch.FloatTensor([0, 1] * (N_sim+int(N_TrainOrg/2))).to(device)
# y2 = torch.FloatTensor([0, 1] * (int(N_Train/2))).to(device)
# y2 = torch.reshape(y2,(-1,1))
# print(X2.shape,y2.shape,type(y2))


### data for testing

In [ ]:
print(type(theta_t))

# testing example input

In [ ]:
# data for testing
global X2t,yt
print(X_2.shape)
Full_flag = 1 # 1 means all dataset are used for testing, 0 means subtracted training data.
if Full_flag:
    X2t=X_2.to(device) # only use intensity information
else:
    Train_num = [Train_Gap*l for l in p_num];
#     print('Train_num:',Train_num)
    full_n = [i for i in range(len(X_2))]
    test_n = [x for x in full_n if (x not in Train_num)]
#     print('Test_num:',test_n)
    X2t = X_2[test_n,:].to(device)
    print('Test dataset:',X2t.shape)
    Ntn = Ntn-len(Train_num)

# training process

In [ ]:
seed = 12345
random.seed(seed)
torch.manual_seed(seed)
N = N_Compents
# N = np.size(theta[0])  # num_samples_per_class
D = 2  # dimensions
C = 2  # num_classes
H = 5 # num_hidden_units
B = 33 # num_bins
S = 0 #bin id
# Ntn= 2 # num_test_classes
Si=0
iternum= 45 if not standard_flag else 15
n_feature =20

global N_Train


nn.Conv1d() applies 1D convolution over the input. nn.Conv1d() expects the input to be of the shape [batch_size, input_channels, signal_length] .
Conv1d — Input 2d
To apply 1D convolution on a 2d input signal, we can do the following. First, we define our input tensor of the size [1, 2, 5] where batch_size = 1, input_channels = 2 , and signal_length = 5 .


You are forgetting the "minibatch dimension", each "1D" sample has indeed two dimensions: the number of channels (7 in your example) and length (10 in your case). However, pytorch expects as input not a single sample, but rather a minibatch of B samples stacked together along the "minibatch dimension".
So a "1D" CNN in pytorch expects a 3D tensor as input: BxCxT. If you only have one signal, you can add a singleton dimension:

 out = model(torch.tensor(X)[None, ...])

In [ ]:
  

class LSTM(nn.Module):
    def __init__(self, input_size=N_Compents,  num_layers=1, hidden_size=128, O_feature=1, BinNum = B):
        super(LSTM,self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = torch.nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        #(seq_len, batch, input_size)
 
        self.fc1 = nn.Sequential(
            nn.Linear(self.hidden_size, O_feature),          
            nn.Sigmoid()
        )
            
        self._create_weights()
        

#=======================================================================
#=======================================================================
    def _create_weights(self, mean=0.0, std=0.05):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean, std) 

#=======================================================================
#=======================================================================

    def forward(self, x):
        In_Lstm = x
#         In_Lstm = torch.reshape(In_Lstm,(-1,1,N))
        In_Lstm = torch.reshape(In_Lstm,(-1,1,self.input_size))
        # In_Lstm--> (batch, seq_len=1, input_size) as batch first;
        h0 = torch.zeros(self.num_layers, In_Lstm.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, In_Lstm.size(0), self.hidden_size).to(device)
        # h0, c0 --> (num_layers, batch_size, hidden_size)
        out, _  = self.lstm(In_Lstm,(h0, c0))
        # out --> (batch_size, sequence_len, hidden_size)
        out  = out[:, -1, :]
        out  = self.fc1(out)
        return out

      
# NUM_LAYERS=3 #2
# H_SIZE =256 

NUM_LAYERS= 1 if not standard_flag else 3
H_SIZE = 64 if not standard_flag else 256

O_FeatCOV=1 #num of classes
model = LSTM(
    input_size = int(N_Compents),
    num_layers = NUM_LAYERS,
    hidden_size = H_SIZE,
    O_feature=O_FeatCOV,
    BinNum = B
)
model.to(device)
# x = torch.randn(BATCH_SIZE, IN_DIM)

In [ ]:
type(N_Compents)

In [ ]:
for name,param in model.named_parameters():
    print(name,type(param), param.size())

# H_SIZE*4 (input gate, forget, )

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('pytorch_total_params:',pytorch_total_params)

# pick two extreme curves for training

In [ ]:
print(y2.t(),y2.shape,len(y2))

In [ ]:
# train NN in each bin
def TrainingProc():
    global X2,y2
#     X2 = torch.zeros(N2 * N_Train, D*B).to(device)
#     y2 = torch.zeros(N2 * N_Train, 1, dtype=torch.float).to(device)    
    
#     for b in range(B):
#         for ix in range(N2):
#             for ic in range(N_Train):
#                 X2[ix+ic*N2,2*b:2*b+2] = X_Extrem[b*N2+ix+ic*N]
#                 y2[ix+ic*N2] = y_Extrem[b*N2+ix+ic*N]
    #print("X2:", tuple(X2.size()))
    #print("y2:", tuple(y2.size()))
    #print('y2.shape',y2.shape)
    
    learning_rate = 2e-3  # 8e-3
    lambda_l2 = 2e-5
    # nn package to create our linear model
    # each Linear module has a weight and bias

#     model = nn.Sequential(
#         nn.Linear(D*B, H),
#         nn.ReLU(),

# #         nn.Linear(H, int(H/C)),
# #         nn.ReLU(),
# #         nn.Linear(H, C),
# #         nn.ReLU(),
#         nn.Linear(H, 1),
#         nn.Sigmoid()
#     )
#     model.to(device)

    # nn package also has different loss functions.
    # we use cross entropy loss for our classification task
    criterion = torch.nn.BCELoss()

    # we use the optim package to apply
    # ADAM for our parameter updates
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2
    #optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate, betas=(0.8, 0.999), eps=1e-08, weight_decay=0) # built-in L2

    # e = 1.  # plotting purpose

    # Training
    dh=display.display("Training",display_id=True)
    for t in range(iternum):
        # Feed forward to get the logits
        y_pred = model(X2)
#         y_pred = model(X_Extrem_x,X_Extrem_y)
#         print(y_pred.shape)
        # Compute the loss and accuracy
        loss = criterion(y_pred, y2)
        predicted = y_pred > 0.5
#         print(y2,predicted)
        acc = (y2 == predicted).sum().float() / len(y2)
        dh.update(" [EPOCH]: %i, [LOSS]: %.6f, [ACCURACY]: %.3f" % ( t, loss.item(), acc))
        #display.clear_output(wait=True)
              
        # zero the gradients before running
        # the backward pass.
        optimizer.zero_grad()
    
        # Backward pass to compute the gradient
        # of loss w.r.t our learnable params. 
        loss.backward()
    
        # Update params
        optimizer.step()
        
    #print(len(y2),y_pred.size(),loss,sep='\n') 
    return acc
#     return acc,model

In [ ]:
# pick two extreme curves for training and generate test data in each bin
global Acc,N2
N2=int(N/B)
ytest_pred=[]
# print(y2.t())
# ====================training=======================
from time import time
t0= time()
Acc =TrainingProc()  
t1= time()
print("training done in %0.3fs" % (t1 - t0))
with torch.no_grad(): 
    y_pred= model(X2) # 

print(y_pred.shape)

In [ ]:
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
with torch.no_grad(): 
    y_pred= model(X2) # 
y_pred = y_pred>0.5
#     y_pred = model(X_Extrem_x,X_Extrem_y)

print(y_pred[0:10].t(),y_pred.shape)

In [ ]:
print(X2)

In [ ]:
print(X2t.shape,X2.shape)
print(X2.shape,X2t.shape,N)

In [ ]:

with torch.no_grad(): 
    y2t = model(X2t)# use model in each bin to predict label of test data curves belong to each bin             
y2t_pred = y2t>0.5 
y2t_pred = torch.squeeze(y2t_pred)
# FinlPredLabel =y2t_pred.cpu().numpy().astype(int)
FinlPredLabel =1*y2t_pred.cpu()
print(y2t_pred.shape)

print(FinlPredLabel.shape,FinlPredLabel)
        

# Curve-wise ambigious region 

In [ ]:
    var_C0=0 # initial value of class 0;
    Ntn=y2t_pred.shape[0]
    for j in range(Ntn-1): # test data sets(curves)    
        if (FinlPredLabel[j] == 0) and (FinlPredLabel[(j+1)]==0):
            var_C0 = j+1
        else:
            break  
    var_C1=Ntn-1 # initial value of class 1; So after iteration, if var_C1=Ntn-1(still this value), it means the following conditions are not met.
    for j in range(Ntn-1,0,-1): # test data sets(curves)   
        if (FinlPredLabel[j] == 1) and (FinlPredLabel[(j-1)]==1):
            var_C1 = j-1
        else:
            break
    torch.set_printoptions(threshold=10_000)
    np.set_printoptions(threshold=10_000)
     
    if var_C0==Ntn-1:
        Glo_acc = 0.50
        print("Interval with low classification confidence is (%i ,  %i)" % (var_C0,  var_C1))
        print("The boundary is not found  and all curves are labeled as 0.")
    elif var_C1==0:
        Glo_acc = 0.50
        print("Interval with low classification confidence is (%i ,  %i)" % (var_C0,  var_C1))
        print("The boundary is not found and all curves are labeled as 1.")
    elif var_C0==0 and var_C1==Ntn-1:
        Glo_acc = 0.00
        print("Interval with low classification confidence is (%i ,  %i)" % (var_C0,  var_C1))
        print("The boundary is not found  and all curves are labeled as 1." )
    else:
        Glo_acc = 1-float((var_C1-var_C0-1)/Ntn)
        print("Interval with low classification confidence is (%i ,  %i)" % (var_C0,  var_C1))
        print("Overall classification accuracy is %.3f" % (Glo_acc*100)+"%")
    

In [ ]:
# print(FinlPredLabel[p0_num*gap])
# print(FinlPredLabel[p_num*gap])
print(FinlPredLabel[p_num[0]*Train_Gap])
print(FinlPredLabel[p_num[1]*Train_Gap])
print(FinlPredLabel[p_num[2]*Train_Gap])
print(FinlPredLabel[p_num[3]*Train_Gap])